In [38]:
## Link to repository - https://github.com/biony1209/DSCI-510-Final-Project

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
covid = pd.read_csv(r'C:\Users\jaimv\Downloads\covid.csv')
noncovid = pd.read_csv(r'C:\Users\jaimv\Downloads\noncovid.csv')

In [7]:
#df = pd.read_csv('../data/covidsample.csv')

In [8]:
covid.rename({'date_died': 'outcome'}, axis=1, inplace=True)

In [9]:
noncovid = noncovid[["ID", "outcome", "age", "gendera", "hypertensive", "diabetes"]]

In [10]:
noncovid.rename({'ID': 'id', 'gendera': 'sex', 'hypertensive': 'hypertension'}, axis=1, inplace=True)

In [11]:
transform={2:1,1:0}
covid = covid.replace({'sex':transform,'hypertension':transform, 'diabetes':transform})

In [12]:
noncovid = noncovid.replace({'sex':transform})

In [13]:
covid['outcome'][covid['outcome']!='9999-99-99']=1.0

C:\Users\jaimv\AppData\Local\Temp/ipykernel_41832/4100060542.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid['outcome'][covid['outcome']!='9999-99-99']=1.0


In [14]:
covid['outcome'][covid['outcome']!=1]=0.0

C:\Users\jaimv\AppData\Local\Temp/ipykernel_41832/1287461197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid['outcome'][covid['outcome']!=1]=0.0


In [15]:
covid = covid[["id", "outcome", "age", "sex", "hypertension", "diabetes"]]

In [16]:
covid['covid_status'] = 1
noncovid['covid_status'] = 0

In [17]:
covid['outcome'] = covid['outcome'].dropna().astype('uint8')

In [18]:
covid

,id,outcome,age,sex,hypertension,diabetes,covid_status
0,16169f,0,27,1,1,1,1
1,1009bf,0,24,1,1,1,1
2,167386,0,54,0,1,1,1
3,0b5948,0,30,1,1,1,1
4,0d01b5,1,60,0,0,0,1
...,...,...,...,...,...,...,...
566597,01ff60,0,58,1,1,0,1
566598,047cd1,0,48,0,1,1,1
566599,1beb81,0,49,0,1,1,1
566600,16fb02,0,43,0,1,1,1


In [19]:
noncovid = noncovid.dropna()
covid = covid.dropna()

In [20]:
noncovid['outcome'] = noncovid['outcome'].astype(int)
covid['outcome'] = covid['outcome'].astype(int)

In [21]:
noncovid

,id,outcome,age,sex,hypertension,diabetes,covid_status
0,125047,0,72,0,0,1,0
1,139812,0,75,1,0,0,0
2,109787,0,83,1,0,0,0
3,130587,0,43,1,0,0,0
4,138290,0,75,1,1,0,0
...,...,...,...,...,...,...,...
1172,171130,0,62,0,1,1,0
1173,101659,0,78,0,0,1,0
1174,162069,0,85,1,1,1,0
1175,120967,0,79,1,0,1,0


In [22]:
full_cohort = [covid, noncovid]

result = pd.concat(full_cohort, ignore_index=True)

In [23]:
result = result[~result['outcome'].isnull()]
result['outcome'] = result['outcome'].astype(int)
result['age'] = result['age'].astype(int)
result['sex'] = result['sex'].astype(int)
result['hypertension'] = result['hypertension'].astype(int)
result['diabetes'] = result['diabetes'].astype(int)
result['covid_status'] = result['covid_status'].astype(int)

In [24]:
result = result[result['outcome'] != 98]
result = result[result['sex'] != 98]
result = result[result['hypertension'] != 98]
result = result[result['diabetes'] != 98]

In [25]:
import statsmodels.formula.api as smf

In [26]:
model=smf.logit(formula='outcome ~ covid_status + age + sex',data=result).fit()
model_odds=pd.DataFrame(np.exp(model.params),columns=['Odds Ratio'])
model_odds['p-value']=model.pvalues
model_odds[['5%','95%']]=np.exp(model.conf_int())
model_odds['significant']=model_odds['p-value']<.05
model_odds=model_odds.drop('Intercept')
model_odds=model_odds.sort_values(by='Odds Ratio',ascending=False)
model_odds

Optimization terminated successfully.
         Current function value: 0.195904
         Iterations 8


,Odds Ratio,p-value,5%,95%,significant
covid_status,2.841336,8.996275e-32,2.386385,3.383022,True
sex,1.807317,0.000000e+00,1.765253,1.850383,True
age,1.071549,0.000000e+00,1.070798,1.072301,True


## Matching

In [27]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import math